# Estimation of overtopping failure probability (Hunt's equation)

In this example, we will demonstrate the application of various reliability methods to estimate the probability of failure of a levee due to overtopping. The failure mechanism is characterized by the Hunt's equation, which is a simple model to estimate the wave runup.

### Define model

First, we import the necessary packages:

In [1]:
from ptk import *
import numpy as np

We consider the following limit state function:

$Z = h_{crest} - (h + R_u)$

where: <br>

$h_{crest}$ is the crest height of the flood defence [m] <br>
$h$ is the water level on the flood defence [m] <br>
$R_u$ represents the wave runup [m]

The wave runup $R_u$ is derived using the Hunt's equation:

$R_u = \xi \cdot H_s$

with:

$\xi = \frac{\tan (\alpha)}{\sqrt{2\cdot \pi \cdot H_s/L_0}} $

$L_0 = g \cdot T_p^2$

where:

$H_s$ is the significant wave height (m) <br>
$T_p$ is the peak wave period (s) <br>
$g$ is the gravitational acceleration ($9.81 m/s ^2$) <br>
$\tan (\alpha)$ is the slope of the flood defence (-)

The parameters $h$, $H_s$ and $T_p$ represent the imposed load, while $h_{crest}$ and $\tan (\alpha)$ stand for the strength of the levee.

In [2]:
def hunt(T_p, tan_alpha, H_s, h_crest, h):
    g = 9.81
    L_0 = g  * T_p * T_p
    xi = tan_alpha / np.sqrt(2 * np.pi * H_s / L_0)
    R_u = xi * H_s

    return h_crest - (h + R_u)

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [3]:
project = ReliabilityProject()
project.model = hunt

We assume the following distributions for the parameters present in the limit state function:

In [4]:
project.variables["T_p"].distribution = "log_normal"
project.variables["T_p"].mean = 6
project.variables["T_p"].deviation = 2

project.variables["tan_alpha"].distribution = "deterministic"
project.variables["tan_alpha"].mean = 0.333333

project.variables["H_s"].distribution = "log_normal"
project.variables["H_s"].mean = 3
project.variables["H_s"].deviation = 1

project.variables["h_crest"].distribution = "log_normal"
project.variables["h_crest"].mean = 10
project.variables["h_crest"].deviation = 0.05

project.variables["h"].distribution = 'exponential'
project.variables["h"].shift = 0.5
project.variables["h"].scale = 1

### Perform reliability calculations with Directional Sampling

We start with the reliability method `Directional Sampling`. We choose the calculation settings: `minimum_directions`, `maximum_directions` and `variation_coefficient`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.design_point`.

In [5]:
project.settings.reliability_method = "directional_sampling"

project.settings.minimum_directions = 10000
project.settings.maximum_directions = 20000
project.settings.variation_coefficient = 0.02

project.run()

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")

read_results(project.design_point)

Beta = 1.8711604829760657
Probability of failure = 0.030661420137601914
T_p: alpha = -0.7599460091669649, x = 9.030827950876963
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.3881938534023303, x = 3.602786268469947
h_crest: alpha = 0.01326241797916879, x = 9.998634296961573
h: alpha = -0.5211637972856513, x = 2.3034158389094634
Converged (convergence = 0.019998996781194274 < 0.02)


### Perform reliability calculations with Importance Sampling

We now conduct the reliability analysis using the `Importance Sampling`. We choose the calculation settings: `minimum_samples`, `maximum_samples` and `variation_coefficient`.

In [6]:
project.settings.reliability_method = "importance_sampling"

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 100000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.878734663894228
Probability of failure = 0.030140366090895945
T_p: alpha = -0.7635028627497121, x = 9.06736272143939
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.3800099068522848, x = 3.5882733586165823
h_crest: alpha = 0.018696192467041448, x = 9.99811893023878
h: alpha = -0.5218297630974563, x = 2.311251988352642
Not converged (convergence = 0.020664889091332777 > 0.02)


### Perform reliability calculations with Adaptive Importance Sampling

We now conduct the reliability analysis using the `Adaptive Importance Sampling` method. We choose the calculation settings: `minimum_samples`, `maximum_samples`, `minimum_variance_loops`, `maximum_variance_loops`, `fraction_failed` and `variation_coefficient`.

In [7]:
project.settings.reliability_method = "adaptive_importance_sampling"

project.settings.minimum_samples = 10000
project.settings.maximum_samples = 100000
project.settings.minimum_variance_loops = 5
project.settings.maximum_variance_loops = 10
project.settings.fraction_failed = 0.5
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.8800015664915952
Probability of failure = 0.030053932215033417
T_p: alpha = -0.7655055789023627, x = 9.08130180954658
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.38014700983128114, x = 3.5891344163464347
h_crest: alpha = 0.008789833715386889, x = 9.999048806910018
h: alpha = -0.519049321749822, x = 2.3043679996051454
Converged (convergence = 0.019998432841262077 < 0.02)


### Perform reliability calculations with Subset Simulation

We now conduct the reliability analysis using the `Subset Simulation` method. We choose the calculation settings: `minimum_samples`, `maximum_samples` and `variation_coefficient`.

In [8]:
project.settings.reliability_method = "subset_simulation"

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 50000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.6340207682230106
Probability of failure = 0.05112724493695072
T_p: alpha = -0.767128840245305, x = 8.550226642518481
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.37864858343136387, x = 3.4790664720049045
h_crest: alpha = 0.03424486534172827, x = 9.997077605092892
h: alpha = -0.5166874122010732, x = 2.113159584596323
Converged (convergence = 0.019993596499694042 < 0.02)
